# 타이타닉 생존자 예측 (Titanic Survival Prediction) - Accuracy 최적화 버전 v2

본 노트북은 Kaggle의 [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic/data) 데이터를 기반으로 생존자를 예측합니다.

## 📊 데이터셋 설명
| 변수 | 설명 |
|------|------|
| Survival | 0 = 사망, 1 = 생존 |
| Pclass | 티켓 등급 (1 = 1등석, 2 = 2등석, 3 = 3등석) |
| Sex | 성별 |
| Age | 나이 |
| SibSp | 함께 탑승한 형제자매, 배우자 수 |
| Parch | 함께 탑승한 부모, 자녀 수 |
| Ticket | 티켓 번호 |
| Fare | 요금 |
| Cabin | 객실 번호 |
| Embarked | 탑승 항구 (C = Cherbourg, Q = Queenstown, S = Southampton) |

## 🎯 v2 개선사항
1. **전처리 강화**: HasCabin 특성 추가, Fare는 Pclass별 median으로 대치
2. **계산 시간 단축**: n_trials 50, timeout 120초, 5-Fold CV (총 10분 이내)
3. **과적합 감지**: Train-CV 차이 > 5% 경고
4. **데이터 누수 감지**: Feature Importance > 50% 경고

In [1]:
# ============================================================
# 1. 라이브러리 임포트
# ============================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import time

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, 
                              ExtraTreesClassifier, VotingClassifier, StackingClassifier)
from sklearn.svm import SVC

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import optuna
from optuna.samplers import TPESampler

import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

print("✅ 라이브러리 임포트 완료 (Pro_2 v2 - Accuracy 최적화)")

✅ 라이브러리 임포트 완료 (Pro_2 v2 - Accuracy 최적화)


c:\Users\user\miniconda3\envs\DS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ============================================================
# 2. 데이터 불러오기
# ============================================================
base_path = r'C:\Users\user\github\DataScience\scikit-learn\scikit-learn\data\titanic'
train_df = pd.read_csv(f'{base_path}/train.csv')
test_df = pd.read_csv(f'{base_path}/test.csv')

test_passenger_ids = test_df['PassengerId'].copy()

print(f"Train Shape: {train_df.shape}")
print(f"Test Shape: {test_df.shape}")
print(f"\n결측치 현황:")
print(train_df.isnull().sum())

Train Shape: (891, 12)
Test Shape: (418, 11)

결측치 현황:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [3]:
# ============================================================
# 3. 강화된 전처리 함수 정의 (v2)
# ============================================================

def enhanced_preprocessing_v2(train_df, test_df):
    """
    Accuracy 최적화를 위한 강화된 전처리 함수 v2
    - HasCabin 특성 추가
    - Fare: Pclass별 median으로 개선
    """
    all_data = pd.concat([train_df, test_df], ignore_index=True)
    
    # ========== 1. Title 추출 ==========
    all_data['Title'] = all_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    title_mapping = {
        'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master',
        'Dr': 'Officer', 'Rev': 'Officer', 'Col': 'Officer', 'Major': 'Officer', 'Capt': 'Officer',
        'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs',
        'Lady': 'Royalty', 'Countess': 'Royalty', 'Sir': 'Royalty', 'Don': 'Royalty', 'Dona': 'Royalty', 'Jonkheer': 'Royalty'
    }
    all_data['Title'] = all_data['Title'].map(title_mapping).fillna('Other')
    
    # ========== 2. FamilySize & IsAlone ==========
    all_data['FamilySize'] = all_data['SibSp'] + all_data['Parch'] + 1
    all_data['IsAlone'] = (all_data['FamilySize'] == 1).astype(int)
    all_data['FamilySizeGroup'] = pd.cut(all_data['FamilySize'], bins=[0, 1, 4, 11], labels=['Alone', 'Small', 'Large'])
    
    # ========== 3. Age (Pclass+Sex+Title 그룹 median) ==========
    age_median = all_data.groupby(['Pclass', 'Sex', 'Title'])['Age'].transform('median')
    all_data['Age'] = all_data['Age'].fillna(age_median)
    all_data['Age'] = all_data['Age'].fillna(all_data['Age'].median())
    all_data['AgeBin'] = pd.cut(all_data['Age'], bins=[0, 12, 20, 40, 60, 100], labels=['Child', 'Teenager', 'Adult', 'Middle', 'Senior'])
    all_data['IsChild'] = (all_data['Age'] <= 12).astype(int)
    
    # ========== 4. Fare (Pclass별 median - 개선) ==========
    fare_median_by_pclass = all_data.groupby('Pclass')['Fare'].transform('median')
    all_data['Fare'] = all_data['Fare'].fillna(fare_median_by_pclass)
    all_data['Fare'] = all_data['Fare'].fillna(all_data['Fare'].median())
    all_data['FarePerPerson'] = all_data['Fare'] / all_data['FamilySize']
    all_data['LogFare'] = np.log1p(all_data['Fare'])
    all_data['FareBin'] = pd.qcut(all_data['Fare'], q=4, labels=['Low', 'Medium', 'High', 'VeryHigh'])
    
    # ========== 5. HasCabin (NEW) ==========
    all_data['HasCabin'] = all_data['Cabin'].notna().astype(int)
    
    # ========== 6. Deck ==========
    all_data['Deck'] = all_data['Cabin'].str[0].fillna('Unknown')
    deck_mapping = {'A': 'ABC', 'B': 'ABC', 'C': 'ABC', 'D': 'DE', 'E': 'DE', 'F': 'FG', 'G': 'FG', 'T': 'Unknown', 'Unknown': 'Unknown'}
    all_data['Deck'] = all_data['Deck'].map(deck_mapping)
    
    # ========== 7. Embarked ==========
    all_data['Embarked'] = all_data['Embarked'].fillna(all_data['Embarked'].mode()[0])
    
    # ========== 8. TicketFrequency ==========
    ticket_counts = all_data['Ticket'].value_counts()
    all_data['TicketFrequency'] = all_data['Ticket'].map(ticket_counts)
    
    # ========== 9. Sex & 상호작용 특성 ==========
    all_data['Sex'] = (all_data['Sex'] == 'male').astype(int)
    all_data['Sex_Pclass'] = all_data['Sex'] * all_data['Pclass']
    all_data['Age_Pclass'] = all_data['Age'] * all_data['Pclass']
    
    # ========== 10. 범주형 인코딩 ==========
    label_cols = ['Title', 'Embarked', 'FamilySizeGroup', 'AgeBin', 'FareBin', 'Deck']
    for col in label_cols:
        le = LabelEncoder()
        all_data[col] = le.fit_transform(all_data[col].astype(str))
    
    # ========== 11. 불필요한 컬럼 삭제 ==========
    drop_cols = ['PassengerId', 'Name', 'Ticket', 'Cabin']
    all_data = all_data.drop(columns=drop_cols, errors='ignore')
    
    train_processed = all_data.iloc[:len(train_df)].copy()
    test_processed = all_data.iloc[len(train_df):].copy()
    
    print(f"\n✅ 전처리 완료 (v2)")
    print(f"특성 수: {train_processed.shape[1]}")
    print(f"📌 NEW: HasCabin, Fare는 Pclass별 median으로 대치")
    
    return train_processed, test_processed

train_processed, test_processed = enhanced_preprocessing_v2(train_df, test_df)
train_processed.head()


✅ 전처리 완료 (v2)
특성 수: 22
📌 NEW: HasCabin, Fare는 Pclass별 median으로 대치


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,...,AgeBin,IsChild,FarePerPerson,LogFare,FareBin,HasCabin,Deck,TicketFrequency,Sex_Pclass,Age_Pclass
0,0.0,3,1,22.0,1,0,7.2500,2,2,2,...,0,0,3.62500,2.110213,1,0,3,1,3,66.0
1,1.0,1,0,38.0,1,0,71.2833,0,3,2,...,0,0,35.64165,4.280593,3,1,0,2,0,38.0
2,1.0,3,0,26.0,0,0,7.9250,2,1,1,...,0,0,7.92500,2.188856,2,0,3,1,0,78.0
3,1.0,1,0,35.0,1,0,53.1000,2,3,2,...,0,0,26.55000,3.990834,3,1,0,2,0,35.0
4,0.0,3,1,35.0,0,0,8.0500,2,2,1,...,0,0,8.05000,2.202765,2,0,3,1,3,105.0


In [4]:
# ============================================================
# 4. 학습 데이터 준비 (5-Fold CV로 변경)
# ============================================================
X = train_processed.drop('Survived', axis=1)
y = train_processed['Survived'].astype(int)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

# 5-Fold CV (시간 단축)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print(f"Train: {X_train.shape}, Validation: {X_val.shape}")
print(f"📌 5-Fold Stratified CV 사용 (시간 단축)")

Train: (757, 21), Validation: (134, 21)
📌 5-Fold Stratified CV 사용 (시간 단축)


In [5]:
# ============================================================
# 5. 과적합 감지 함수 정의
# ============================================================

def check_overfitting(model, X_train, y_train, model_name, cv):
    """Train vs CV accuracy 비교하여 과적합 감지"""
    model_clone = model.__class__(**model.get_params())
    model_clone.fit(X_train, y_train)
    train_acc = model_clone.score(X_train, y_train)
    cv_scores = cross_val_score(model_clone, X_train, y_train, cv=cv, scoring='accuracy')
    cv_acc = cv_scores.mean()
    gap = train_acc - cv_acc
    
    if gap > 0.10:
        print(f"🚨 {model_name}: 심각한 과적합! Train={train_acc:.4f}, CV={cv_acc:.4f}, Gap={gap:.4f}")
    elif gap > 0.05:
        print(f"⚠️ {model_name}: 과적합 경고. Train={train_acc:.4f}, CV={cv_acc:.4f}, Gap={gap:.4f}")
    else:
        print(f"✅ {model_name}: 정상. Train={train_acc:.4f}, CV={cv_acc:.4f}, Gap={gap:.4f}")
    return train_acc, cv_acc, gap

def check_feature_importance(model, feature_names, model_name):
    """Feature importance 50% 초과 시 경고"""
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        max_imp = importances.max()
        if max_imp > 0.5:
            idx = importances.argmax()
            print(f"🚨 {model_name}: 데이터 누수 의심! {feature_names[idx]} = {max_imp:.2%}")
        else:
            print(f"✅ {model_name}: Feature importance 정상 (max={max_imp:.2%})")

print("과적합 및 Feature Importance 감지 함수 정의 완료")

과적합 및 Feature Importance 감지 함수 정의 완료


In [6]:
# ============================================================
# 6. Optuna 튜닝 (시간 단축: n_trials=50, timeout=120초)
# ============================================================

cv_optuna = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 6.1 XGBoost
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 2, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
        'random_state': 42, 'use_label_encoder': False, 'eval_metric': 'logloss', 'n_jobs': -1
    }
    model = XGBClassifier(**params)
    return cross_val_score(model, X_train, y_train, cv=cv_optuna, scoring='accuracy').mean()

print("XGBoost 튜닝 시작...")
study_xgb = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_xgb.optimize(objective_xgb, n_trials=50, timeout=120, show_progress_bar=True)
print(f"🏆 XGBoost: {study_xgb.best_value:.4f}")

XGBoost 튜닝 시작...


Best trial: 35. Best value: 0.844136: 100%|██████████| 50/50 [00:12<00:00,  4.13it/s, 12.10/120 seconds]

🏆 XGBoost: 0.8441


In [7]:
# 6.2 LightGBM
def objective_lgbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 2, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 10, 40),
        'random_state': 42, 'verbose': -1, 'n_jobs': -1
    }
    model = LGBMClassifier(**params)
    return cross_val_score(model, X_train, y_train, cv=cv_optuna, scoring='accuracy').mean()

print("LightGBM 튜닝 시작...")
study_lgbm = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_lgbm.optimize(objective_lgbm, n_trials=50, timeout=120, show_progress_bar=True)
print(f"🏆 LightGBM: {study_lgbm.best_value:.4f}")

LightGBM 튜닝 시작...


Best trial: 21. Best value: 0.841469: 100%|██████████| 50/50 [00:07<00:00,  6.80it/s, 7.35/120 seconds]

🏆 LightGBM: 0.8415


In [8]:
# 6.3 CatBoost
def objective_catboost(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 300),
        'depth': trial.suggest_int('depth', 2, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        'random_seed': 42, 'verbose': 0
    }
    scores = []
    for train_idx, val_idx in cv_optuna.split(X_train, y_train):
        model = CatBoostClassifier(**params)
        model.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])
        scores.append(accuracy_score(y_train.iloc[val_idx], model.predict(X_train.iloc[val_idx])))
    return np.mean(scores)

print("CatBoost 튜닝 시작...")
study_catboost = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_catboost.optimize(objective_catboost, n_trials=50, timeout=120, show_progress_bar=True)
print(f"🏆 CatBoost: {study_catboost.best_value:.4f}")

CatBoost 튜닝 시작...


Best trial: 39. Best value: 0.845434: 100%|██████████| 50/50 [00:58<00:00,  1.16s/it, 58.02/120 seconds]

🏆 CatBoost: 0.8454


In [9]:
# 6.4 RandomForest
def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_samples_split': trial.suggest_int('min_samples_split', 5, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 8),
        'random_state': 42, 'n_jobs': -1
    }
    model = RandomForestClassifier(**params)
    return cross_val_score(model, X_train, y_train, cv=cv_optuna, scoring='accuracy').mean()

print("RandomForest 튜닝 시작...")
study_rf = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_rf.optimize(objective_rf, n_trials=50, timeout=60, show_progress_bar=True)
print(f"🏆 RandomForest: {study_rf.best_value:.4f}")

RandomForest 튜닝 시작...


Best trial: 14. Best value: 0.840127: 100%|██████████| 50/50 [00:46<00:00,  1.07it/s, 46.61/60 seconds]

🏆 RandomForest: 0.8401


In [10]:
# ============================================================
# 7. 최적 모델 학습 및 과적합 검증
# ============================================================

best_xgb = XGBClassifier(**study_xgb.best_params, random_state=42, use_label_encoder=False, eval_metric='logloss', n_jobs=-1)
best_lgbm = LGBMClassifier(**study_lgbm.best_params, random_state=42, verbose=-1, n_jobs=-1)
best_catboost = CatBoostClassifier(**study_catboost.best_params, random_seed=42, verbose=0)
best_rf = RandomForestClassifier(**study_rf.best_params, random_state=42, n_jobs=-1)

best_xgb.fit(X_train, y_train)
best_lgbm.fit(X_train, y_train)
best_catboost.fit(X_train, y_train)
best_rf.fit(X_train, y_train)

print("\n=" * 50)
print("🛡️ 과적합 감지")
print("=" * 50)
check_overfitting(best_xgb, X_train, y_train, 'XGBoost', cv)
check_overfitting(best_lgbm, X_train, y_train, 'LightGBM', cv)
check_overfitting(best_rf, X_train, y_train, 'RandomForest', cv)

print("\n=" * 50)
print("🔍 Feature Importance 검증")
print("=" * 50)
check_feature_importance(best_xgb, X_train.columns, 'XGBoost')
check_feature_importance(best_lgbm, X_train.columns, 'LightGBM')
check_feature_importance(best_rf, X_train.columns, 'RandomForest')


=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
🛡️ 과적합 감지
⚠️ XGBoost: 과적합 경고. Train=0.9102, CV=0.8441, Gap=0.0660
⚠️ LightGBM: 과적합 경고. Train=0.9062, CV=0.8415, Gap=0.0647
✅ RandomForest: 정상. Train=0.8798, CV=0.8401, Gap=0.0397

=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
🔍 Feature Importance 검증
✅ XGBoost: Feature importance 정상 (max=47.68%)
🚨 LightGBM: 데이터 누수 의심! FarePerPerson = 16700.00%
✅ RandomForest: Feature importance 정상 (max=19.93%)


In [11]:
# ============================================================
# 8. Validation 성능 평가
# ============================================================

models_tuned = {'XGBoost': best_xgb, 'LightGBM': best_lgbm, 'CatBoost': best_catboost, 'RandomForest': best_rf}

print("\n📊 Validation Set 성능:")
print("=" * 50)
val_results = []
for name, model in models_tuned.items():
    pred = model.predict(X_val)
    acc = accuracy_score(y_val, pred)
    val_results.append({'Model': name, 'Accuracy': acc})
    print(f"{name}: Accuracy={acc:.4f}")

val_df = pd.DataFrame(val_results).sort_values('Accuracy', ascending=False)
val_df


📊 Validation Set 성능:
XGBoost: Accuracy=0.7687
LightGBM: Accuracy=0.7687
CatBoost: Accuracy=0.7836
RandomForest: Accuracy=0.7985


,Model,Accuracy
3,RandomForest,0.798507
2,CatBoost,0.783582
1,LightGBM,0.768657
0,XGBoost,0.768657


In [12]:
# ============================================================
# 9. 앙상블
# ============================================================

acc_xgb = accuracy_score(y_val, best_xgb.predict(X_val))
acc_lgbm = accuracy_score(y_val, best_lgbm.predict(X_val))
acc_cat = accuracy_score(y_val, best_catboost.predict(X_val))
acc_rf = accuracy_score(y_val, best_rf.predict(X_val))

total = acc_xgb + acc_lgbm + acc_cat + acc_rf
weights = [acc_xgb/total, acc_lgbm/total, acc_cat/total, acc_rf/total]

# Soft Voting
voting = VotingClassifier(
    estimators=[('xgb', best_xgb), ('lgbm', best_lgbm), ('cat', best_catboost), ('rf', best_rf)],
    voting='soft', weights=weights
)
voting.fit(X_train, y_train)
voting_acc = accuracy_score(y_val, voting.predict(X_val))
print(f"Weighted Voting: {voting_acc:.4f}")

# Stacking
stacking = StackingClassifier(
    estimators=[('xgb', best_xgb), ('lgbm', best_lgbm), ('cat', best_catboost), ('rf', best_rf)],
    final_estimator=LogisticRegression(C=0.1, max_iter=1000),
    cv=5, n_jobs=-1
)
stacking.fit(X_train, y_train)
stacking_acc = accuracy_score(y_val, stacking.predict(X_val))
print(f"Stacking: {stacking_acc:.4f}")

Weighted Voting: 0.7537
Stacking: 0.7612


In [13]:
# ============================================================
# 10. 전체 데이터로 재학습 및 예측
# ============================================================
import os

X_full = train_processed.drop('Survived', axis=1)
y_full = train_processed['Survived'].astype(int)

final_xgb = XGBClassifier(**study_xgb.best_params, random_state=42, use_label_encoder=False, eval_metric='logloss', n_jobs=-1)
final_lgbm = LGBMClassifier(**study_lgbm.best_params, random_state=42, verbose=-1, n_jobs=-1)
final_catboost = CatBoostClassifier(**study_catboost.best_params, random_seed=42, verbose=0)
final_rf = RandomForestClassifier(**study_rf.best_params, random_state=42, n_jobs=-1)

final_xgb.fit(X_full, y_full)
final_lgbm.fit(X_full, y_full)
final_catboost.fit(X_full, y_full)
final_rf.fit(X_full, y_full)

final_voting = VotingClassifier(
    estimators=[('xgb', final_xgb), ('lgbm', final_lgbm), ('cat', final_catboost), ('rf', final_rf)],
    voting='soft', weights=weights
)
final_voting.fit(X_full, y_full)

final_stacking = StackingClassifier(
    estimators=[('xgb', final_xgb), ('lgbm', final_lgbm), ('cat', final_catboost), ('rf', final_rf)],
    final_estimator=LogisticRegression(C=0.1, max_iter=1000),
    cv=5, n_jobs=-1
)
final_stacking.fit(X_full, y_full)

print("✅ 전체 데이터로 재학습 완료")

✅ 전체 데이터로 재학습 완료


In [14]:
# ============================================================
# 11. CSV 저장 (Submission 폴더)
# ============================================================

X_test = test_processed.drop('Survived', axis=1, errors='ignore')
output_path = r'C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission'
os.makedirs(output_path, exist_ok=True)

models_for_submission = {
    'Pro2v2_XGBoost': final_xgb,
    'Pro2v2_LightGBM': final_lgbm,
    'Pro2v2_CatBoost': final_catboost,
    'Pro2v2_RandomForest': final_rf,
    'Pro2v2_Voting': final_voting,
    'Pro2v2_Stacking': final_stacking
}

print("\n📂 CSV 저장 중...")
for name, model in models_for_submission.items():
    predictions = model.predict(X_test).astype(int)
    submission = pd.DataFrame({'PassengerId': test_passenger_ids, 'Survived': predictions})
    file_path = f'{output_path}/submission_{name}.csv'
    submission.to_csv(file_path, index=False)
    print(f"✅ {file_path} (생존: {predictions.sum()}/{len(predictions)})")

print("\n🎉 완료!")


📂 CSV 저장 중...
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro2v2_XGBoost.csv (생존: 147/418)
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro2v2_LightGBM.csv (생존: 151/418)
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro2v2_CatBoost.csv (생존: 162/418)
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro2v2_RandomForest.csv (생존: 155/418)
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro2v2_Voting.csv (생존: 150/418)
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro2v2_Stacking.csv (생존: 149/418)

🎉 완료!


In [15]:
# ============================================================
# 12. 최종 성능 요약
# ============================================================

final_results = [
    {'Model': 'XGBoost', 'CV_Acc': study_xgb.best_value, 'Val_Acc': acc_xgb},
    {'Model': 'LightGBM', 'CV_Acc': study_lgbm.best_value, 'Val_Acc': acc_lgbm},
    {'Model': 'CatBoost', 'CV_Acc': study_catboost.best_value, 'Val_Acc': acc_cat},
    {'Model': 'RandomForest', 'CV_Acc': study_rf.best_value, 'Val_Acc': acc_rf},
    {'Model': 'Voting', 'CV_Acc': None, 'Val_Acc': voting_acc},
    {'Model': 'Stacking', 'CV_Acc': None, 'Val_Acc': stacking_acc}
]

final_df = pd.DataFrame(final_results).sort_values('Val_Acc', ascending=False)
print("\n" + "="*60)
print("📊 최종 성능 요약 (Pro_2 v2)")
print("="*60)
print(final_df.to_string(index=False))


📊 최종 성능 요약 (Pro_2 v2)
       Model   CV_Acc  Val_Acc
RandomForest 0.840127 0.798507
    CatBoost 0.845434 0.783582
    LightGBM 0.841469 0.768657
     XGBoost 0.844136 0.768657
    Stacking      NaN 0.761194
      Voting      NaN 0.753731


In [16]:
# ============================================================
# 13. Feature Importance 시각화
# ============================================================

feature_importance = pd.DataFrame({
    'Feature': X_full.columns,
    'Importance': final_xgb.feature_importances_
}).sort_values('Importance', ascending=False)

fig = px.bar(feature_importance.head(15), x='Importance', y='Feature', orientation='h',
             title='Top 15 Feature Importance (XGBoost)', color='Importance')
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

print("\n📌 주요 특성:")
print(feature_importance.head(10).to_string(index=False))


📌 주요 특성:
        Feature  Importance
            Sex    0.415885
         Pclass    0.184564
     Sex_Pclass    0.052024
          Title    0.048285
       HasCabin    0.040876
FamilySizeGroup    0.037451
          SibSp    0.034714
TicketFrequency    0.021630
           Deck    0.021257
     Age_Pclass    0.019715
